In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/MNIST"

In [ ]:
## load package
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# load build-in dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
## reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = y_train.reshape(60000,)
y_test = y_test.reshape(10000,)


In [ ]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

In [ ]:
train_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_train]
train_arr_75 = np.array(train_arr_75)
train_arr_75.shape

In [ ]:
test_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_test]
test_arr_75 = np.array(test_arr_75)
test_arr_75.shape

In [ ]:
x_split_train, x_split_validation, y_split_train, y_split_validation = train_test_split(train_arr_75,y_train_one_hot, train_size=0.8, random_state=1)

In [ ]:
plt.imshow(x_split_train[0])

In [ ]:
## define the model architecture

model = Sequential()
model.add(tf.keras.applications.VGG19(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.layers[0].trainable=False

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=opt,
      metrics=[tf.keras.metrics.CategoricalAccuracy()]
  )

In [ ]:
## Trainning the model
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3)
with tf.device('/device:GPU:0'):
  history1 = model.fit(x_split_train, y_split_train, validation_data=(x_split_validation, y_split_validation), epochs = 10, verbose=1, callbacks=[reduce_lr])

In [ ]:
stage_1_lr = history1.history['lr']
stage_1_lr = np.array(stage_1_lr)

In [ ]:
model.save('epoch10_stage_1_MNIST_VGG_build_model.h5')

In [ ]:
np.savetxt('epoch10_stage_1_lr.txt', stage_1_lr, fmt='%f')

In [ ]:
def loss_diagnostics(history):
    # plot loss
    plt.plot(history.history['loss'], color='blue', label='Train')
    plt.plot(history.history['val_loss'], color='orange', label='Validation')
    plt.legend()


In [ ]:
def accuracy_diagnostics(history):
    plt.plot(history.history['categorical_accuracy'], color='blue', label='Train')
    plt.plot(history.history['val_categorical_accuracy'], color='orange', label='Validation')
    plt.legend()

In [ ]:
history1.history.keys()

In [ ]:
accuracy_diagnostics(history1)

In [ ]:
loss_diagnostics(history1)